In [1]:
from AsyncScraper import runFleetScrape
from TRScraper import scrapeTR
from dataScraper import runSailorData

from fullfile import adjust_race_id, main, postCalcAdjust, uploadSailors
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector
import numpy as np

from openskill.models import PlackettLuce, BradleyTerryFull
from regions import teamRegions

In [2]:

calc_all = True

doScrape = False
doUpload = True

targetSeasons = ['f25']
targetTRSeasons = ['s25']

model = PlackettLuce(beta=25.0/120.0)
targetElo = 1000

if doScrape:
    df_races = runFleetScrape()
else:
    # if running scrapers seperately
    df_races = pd.read_json("racesfr.json")

df_races['raceNum'] = df_races['raceID'].apply(
    lambda id: int(id.split("/")[2][:-1]))  # Numeric part
df_races['raceDiv'] = df_races['raceID'].apply(
    lambda id: id.split("/")[2][-1])  # Division part (e.g., 'A', 'B')
df_races['adjusted_raceID'] = df_races.apply(
    adjust_race_id, axis=1)  # to make combined division combined
df_races['Link'] = df_races['Link'].fillna('Unknown')  # fill empty links
# df_races['key'] = np.where(df_races['Link'] == 'Unknown', df_races['Sailor'], df_races['Link'])
df_races['key'] = df_races.apply(
    lambda row: row['Sailor'] + "-" +
    row['Team'] if row['Link'] == 'Unknown' else row['Link'],
    axis=1
)

df_races['partnerKey'] = df_races.apply(
    lambda row: row['Partner'] + "-" +
    row['Team'] if row['PartnerLink'] == 'Unknown' else row['PartnerLink'],
    axis=1
)

if doScrape:
    df_races_tr, trSailors = scrapeTR()
else:
    # if running scrapers seperately
    df_races_tr = pd.read_json("racesTR.json")

df_races_tr['adjusted_raceID'] = df_races_tr['raceID']
df_races_tr['Scoring'] = 'team'
df_races = df_races.rename(
    {'Date': 'date', 'Regatta': 'regatta'}, axis='columns')
df_races_full = pd.concat([df_races, df_races_tr])

# clean up memory
del df_races, df_races_tr

df_races_full = df_races_full.sort_values(
    ['date', 'raceNum', 'raceDiv']).reset_index(drop=True)

df_sailor_ratings = None
if not calc_all:
    cutoff = (datetime.now() - timedelta(weeks=2))
    df_races_full = df_races_full.loc[df_races_full['date'] > cutoff]
    df_sailor_ratings = pd.read_json("sailors-latest.json")      
    # df_sailor_ratings = pd.read_json("sailors-20250424.json")

if doScrape:
    df_sailor_info = runSailorData()
else:
    df_sailor_info = pd.read_json("sailor_data2.json")


In [3]:
people, df_races_full = main(df_sailor_ratings, df_sailor_info, df_races_full)
people, df_sailors, df_races_full = postCalcAdjust(people, df_races_full)

Currently analyzing race 1000/54257 in s16/ike-geiger-team-race, Date:2016-03-19 00:00:00
Currently analyzing race 2000/54257 in s16/oberg, Date:2016-04-16 00:00:00
Currently analyzing race 3000/54257 in f16/stu-nelson, Date:2016-09-17 00:00:00
Currently analyzing race 4000/54257 in f16/protest, Date:2016-10-09 00:00:00
Currently analyzing race 5000/54257 in s17/barnyard-bizzare, Date:2017-03-04 00:00:00
Currently analyzing race 6000/54257 in s17/admiral-moore-team-race, Date:2017-03-25 00:00:00
Currently analyzing race 7000/54257 in s17/new-england-team-race-fowle, Date:2017-04-08 00:00:00
Currently analyzing race 8000/54257 in s17/sperry-women-west-semis, Date:2017-05-23 00:00:00
Currently analyzing race 9000/54257 in f17/owlapalooza-2017, Date:2017-09-30 00:00:00
Currently analyzing race 10000/54257 in f17/nickerson-2017, Date:2017-10-28 00:00:00
Currently analyzing race 11000/54257 in s18/bob-bavier-team-race, Date:2018-03-03 00:00:00
Currently analyzing race 12000/54257 in s18/sea

/Users/carter/Documents/GitHub/sailing-rating-calculation/fullfile.py:869: RuntimeWarning: Mean of empty slice.
  [r['ratio'] for r in p.races if r['pos'] == 'Skipper' and 'ratio' in r.keys()]).mean())
/Users/carter/.pyenv/versions/3.13.1/lib/python3.13/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/carter/Documents/GitHub/sailing-rating-calculation/fullfile.py:871: RuntimeWarning: Mean of empty slice.
  [r['ratio'] for r in p.races if r['pos'] == 'Crew' and 'ratio' in r.keys()]).mean())


16318


In [12]:
# Create a connection
connection = mysql.connector.connect(
    host="localhost",
    port=3308,
    user="root",
    password="password",
    database="crowsnest",
    allow_local_infile=True
)
cursor = connection.cursor()

In [5]:
def uploadTeams(df_sailors, df_races_full, people, cursor, connection):

    def getCounts(races):
        # season_counts = defaultdict(int)
        season_counts = {}

        for race in races:
            season = race["raceID"].split("/")[0]
            if season not in season_counts.keys():
                season_counts[season] = {}
            if race['pos'] not in season_counts[season].keys():
                season_counts[season][race['pos']] = 0
            season_counts[season][race['pos']] += 1

        return dict(season_counts)

    # batch = db.batch()
    # col = db.collection('eloTeams')
    teams = []
    predteams = []
    scrape = False
    teamNames = teamRegions.keys()
    lenteams = len(teamNames)

    season_mask = df_sailors['Seasons'].apply(lambda x: not set(x['skipper']).isdisjoint(
        targetSeasons) or not set(x['crew']).isdisjoint(targetSeasons))

    # Explode the Teams column to enable grouping
    df_exploded = df_sailors.loc[season_mask].explode('Teams')

    # Group by team and compute necessary aggregates
    team_stats = df_exploded.groupby('Teams').agg(
        numCurMembers=('Teams', 'count'),
        avgSkipperOrdinal=('srOrd', 'mean'),
        avgCrewOrdinal=('crOrd', 'mean'),
        avgSkipperRatio=('skipperAvgRatio', 'mean'),
        avgCrewRatio=('crewAvgRatio', 'mean')
    )

    # Calculate the average values as in the original code
    team_stats['avg'] = (team_stats['avgSkipperOrdinal'] +
                         team_stats['avgCrewOrdinal']) / 2
    team_stats['avgRatio'] = (
        team_stats['avgSkipperRatio'] + team_stats['avgCrewRatio']) / 2

    team_stats = team_stats.reindex(teamNames, fill_value=0)
    today = datetime.today()

    # team_link_map = df_races_full.dropna().drop_duplicates('Team').set_index('Team')['Teamlink'].to_dict()

    df_cleaned = df_races_full.dropna(
        subset=['Team', 'Teamlink']).drop_duplicates(subset='Team', keep='first')

    # Create a dictionary with 'Team' as the key and 'TeamLink' as the value
    team_link_map = pd.Series(
        df_cleaned.Teamlink.values, index=df_cleaned.Team).to_dict()


    for i, (team, row) in enumerate(team_stats.iterrows()):
        # if team != "MIT":
        #     continue
        print(f"{i}/{len(team_stats)} {team}")
        avg = row['avg']
        avgRatio = row['avgRatio']
        numCurMembers = row['numCurMembers']

        region = teamRegions[team]
        # teamLink = df_races.loc[df_races['Team'] == team, 'Teamlink'].iloc[0]
        # Default to '' if team not found
        teamLink = team_link_map.get(team, '')
        url = f"https://scores.collegesailing.org/schools/{teamLink.split("/")[2]}"

        filtered_people = [p for p in people.values() if team in p.teams]

        members = [{"name": str(p.name),
                    "key": p.key,
                    "gender": p.gender,
                    "year": str(p.year),
                    'teams': list(p.teams),
                    'skipperRating': int(p.sr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    # 'sr': {'mu': p.sr.mu, 'sigma': p.sr.sigma},
                    'crewRating': int(p.cr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    'womenSkipperRating': int(p.wsr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    'womenCrewRating': int(p.wcr.ordinal(target=targetElo, alpha=200 / model.sigma)),
                    'tsr': p.tsr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'wtsr': p.wtsr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'tcr': p.tcr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'wtcr': p.wtcr.ordinal(target=targetElo, alpha=200 / model.sigma),
                    'avgSkipperRatio': float(p.avgSkipperRatio),
                    'avgCrewRatio': float(p.avgCrewRatio),
                    'raceCount': getCounts(p.races),
                    'seasons': {'skipper': list(p.seasons['skipper']), 'crew': list(p.seasons['crew'])},
                    'cross': sum([race['cross'] for race in p.races if 'cross' in race.keys()]),
                    'outLinks': sum([race['outLinks'] for race in p.races if 'outLinks' in race.keys()]),
                    'skipperRank': int(p.skipperRank),
                    'crewRank': int(p.crewRank),
                    'womenSkipperRank': int(p.womenSkipperRank),
                    'womenCrewRank': int(p.womenCrewRank)
                    } for p in filtered_people]
        print("got members")
        

        topRating = 0
        topWomenRating = 0
        topRatingTR = 0
        topWomenRatingTR = 0

        # '#1': ,'#2': ,
        # teams.append({"name": team,
        #               'topRating': topRating,
        #               'topWomenRating': topWomenRating,
        #               'topRatingTR': topRatingTR,
        #               'topWomenRatingTR': topWomenRatingTR,
        #               #   'teamRating': teamRating,
        #               "avg": avg,
        #               'avgRatio': avgRatio,
        #               "region": region,
        #               "link": url,
        #               'memberCount': numCurMembers,
        #               })

        # predteams.append({"name": team,
        #                   'link': url,
        #                   'topRating': topRating,
        #                   'topRatingTR': topRatingTR,
        #                   'topSkippers': topSkippers,
        #                   'topCrews': topCrews,
        #                   'SkippersTR': topSkippersTR,
        #                   'CrewsTR': topCrewsTR,
        #                   'WomenSkippersTR': topWomenSkippersTR,
        #                   'WomenCrewsTR': topWomenCrewsTR,
        #                   })

        # col.document(team.replace(" ", "-").replace("/", "-").lower()).set({"name": team,
        #                                                                     "avg": avg,
        #                                                                     'topSkippers': topSkippers,
        #                                                                     'topSkippersTR': topSkippersTR,
        #                                                                     'topCrewsTR': topCrewsTR,
        #                                                                     'topRating': topRating,
        #                                                                     'topWomenRating': topWomenRating,
        #                                                                     'topRatingTR': topRatingTR,
        #                                                                     'topWomenRatingTR': topWomenRatingTR,
        #                                                                     'avgRatio': avgRatio,
        #                                                                     "region": region,
        #                                                                     'recentRegattas': recentRegattas,
        #                                                                     "link": url,
        #                                                                     'members': members})

        cursor.execute("""
                    INSERT INTO Teams (teamID, teamName, topFleetRating, topWomenRating, topTeamRating, topWomenTeamRating, avgRatio, region, link)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,   (team, team, topRating, topWomenRating, topRatingTR, topWomenRatingTR, avgRatio, region, url))            
        
        print("inserted team", team)
        for member in members:
            if member['key'] is None:
                print("No key for", member['name'])
                continue
            try:
                cursor.execute("""
                        INSERT IGNORE INTO SailorTeams(sailorID, teamID)
                        VALUES(%s,%s)""", (member['key'].replace("/","-"), 
                                        team))
                # print("Inserted " + member['key'])
            except mysql.connector.errors.IntegrityError as e:
                print(member['key'], "key failed to insert")
                # continue
                raise e
        connection.commit()
        
    #     if i % 20 == 0:  # commit every 20 documents
    #         batch.commit()

    # batch.commit()
    # doc = db.collection('vars').document('eloTeams').set({"teams": teams})

    # doc = db.collection('vars').document('predTeams').set({"teams": predteams})
    # newTeams = sorted(teams, key=lambda x: x['topRating'], reverse=True)
    # return newTeams

In [ ]:
import tempfile
import os

def uploadScores(df_races_full, cursor, connection):
    df_races_full = df_races_full.where(pd.notnull(df_races_full), None)

    # 2️⃣ Filter and preprocess (match your previous logic)
    df_filtered = df_races_full[
        (df_races_full['Scoring'] != 'team') & (~pd.isna(df_races_full['Score']))
    ].copy()

    # 3️⃣ Add computed columns
    df_filtered["season"] = df_filtered["adjusted_raceID"].apply(lambda x: str(x).split("/")[0])
    df_filtered["regatta"] = df_filtered["adjusted_raceID"].apply(lambda x: str(x).split("/")[1])
    df_filtered["sailorID"] = df_filtered.apply(
        lambda r: r["Link"] if r["Link"] != "Unknown" else f"{r['Sailor']}-{r['Team']}", axis=1
    )

    # Select and reorder columns to match the table
    df_sql = df_filtered[
        [
            "season", "regatta", "raceNum", "raceDiv",
            "sailorID", "PartnerLink", "Score",
            "Position", "date"
        ]
    ].copy()

    # Add a None column for `penalty`
    df_sql["penalty"] = None

    # Reorder to match your table definition exactly
    df_sql = df_sql[
        [
            "season", "regatta", "raceNum", "raceDiv",
            "sailorID", "PartnerLink", "Score",
            "penalty", "Position", "date"
        ]
    ]

    # 5️⃣ Write to a temporary CSV
    with tempfile.NamedTemporaryFile(mode="w", suffix=".csv", delete=False) as tmp:
        csv_path = tmp.name
        df_sql.to_csv(tmp, index=False, header=False)

    try:
        cursor.execute("SET GLOBAL local_infile = 1;")
        load_query = f"""
            LOAD DATA LOCAL INFILE '{csv_path.replace("\\", "/")}'
            INTO TABLE FleetScores
            FIELDS TERMINATED BY ','
            OPTIONALLY ENCLOSED BY '"'
            LINES TERMINATED BY '\n'
            (season, regatta, raceNumber, division, sailorID, partnerID, score, penalty, position, date)
        """
        cursor.execute(load_query)
        connection.commit()

        print(f"Successfully inserted {len(df_sql)} rows into Scores.")
    finally:
        os.remove(csv_path)



In [14]:
if doUpload:
    uploadSailors(people, cursor)
    connection.commit() 
    teams = uploadTeams(df_sailors, df_races_full, people, cursor, connection)
    uploadScores(df_races_full, cursor, connection)
    # uploadTops(people)
    # uploadAllSailors(people)
    # connection.commit() 

16318
Currently uploading: 0 Allison Gilson
Currently uploading: 100 Conlan Ridgeway
Currently uploading: 200 Aaron Babier
Currently uploading: 300 Vivianna Sophia Lane
Currently uploading: 400 Peter Hall
Currently uploading: 500 Cara Brickhouse
Currently uploading: 600 Naitik Gupta
Currently uploading: 700 Blake Salisbury
Currently uploading: 800 Remy Margerum
Currently uploading: 900 Gianna Caroniti
Currently uploading: 1000 Juan Moya
Currently uploading: 1100 Linor Rezin
Currently uploading: 1200 Carolyn Applebaum
Currently uploading: 1300 Mya Tajudeen
Currently uploading: 1400 Adrian Salamon
Currently uploading: 1500 Alix Hahn
Currently uploading: 1600 Jackson Eshelman
Currently uploading: 1700 Tyler Mendes
Currently uploading: 1800 Karoline Tyrrell
Currently uploading: 1900 Carly Wilfahrt
Currently uploading: 2000 Linnea Jackson
Currently uploading: 2100 Robert Silcox
Currently uploading: 2200 Young Jeong
Currently uploading: 2300 Halie OBrien
Currently uploading: 2400 Cade Bogusl

In [11]:
cursor.close()
connection.close()

In [ ]:
df_races_full